In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt
import cv2
import requests
import zipfile
import io
from sklearn.metrics import confusion_matrix

# Constants
INPUT_SHAPE = (256, 256, 1)
GITHUB_REPO = "https://github.com/Safwan-Ul-Islam/CSE465_Spring2025_Group-5/tree/main"  # Replace with your GitHub repo
MODEL_URL = f"{GITHUB_REPO}/raw/main/Final%20project/best_model.h5"  # Path to model in repo
TEST_DATA_URL = f"{GITHUB_REPO}/archive/main.zip"  # Will download entire repo as zip
LOCAL_DIR = "./test_data"
TEST_IMAGE_DIR = os.path.join(LOCAL_DIR, "Final project/dataset/test_set/images")  # Adjust paths based on your repo structure
TEST_MASK_DIR = os.path.join(LOCAL_DIR, "Final project/dataset/dataset/test_set/masks")  # Adjust paths based on your repo structure
NUM_SAMPLES_TO_VISUALIZE = 5

# Download helper function
def download_from_github(url, save_path):
    response = requests.get(url)
    if response.status_code == 200:
        with open(save_path, 'wb') as f:
            f.write(response.content)
    else:
        raise Exception(f"Failed to download from {url}")

# Download and extract dataset
def setup_test_data():
    if not os.path.exists(LOCAL_DIR):
        os.makedirs(LOCAL_DIR)

    # Download the zip file
    print("Downloading test data from GitHub...")
    response = requests.get(TEST_DATA_URL)
    zip_file = zipfile.ZipFile(io.BytesIO(response.content))
    zip_file.extractall(LOCAL_DIR)

    # The extracted folder might have a different name (like repo-main)
    # You might need to adjust paths here based on your actual repo structure
    print("Test data downloaded and extracted.")

# Metrics (same as before)
def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def iou_coef(y_true, y_pred, smooth=1):
    intersection = K.sum(K.abs(y_true * y_pred), axis=[1,2,3])
    union = K.sum(y_true,[1,2,3])+K.sum(y_pred,[1,2,3])-intersection
    return K.mean((intersection + smooth) / (union + smooth), axis=0)

def calculate_metrics(y_true, y_pred):
    y_pred_thresh = (y_pred > 0.5).astype(np.float32)
    y_true_f = y_true.flatten()
    y_pred_f = y_pred_thresh.flatten()
    tn, fp, fn, tp = confusion_matrix(y_true_f, y_pred_f).ravel()

    accuracy = (tp + tn) / (tp + tn + fp + fn)
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    dice = (2 * tp) / (2 * tp + fp + fn)
    iou = tp / (tp + fp + fn)

    return {
        'accuracy': accuracy,
        'sensitivity': sensitivity,
        'specificity': specificity,
        'dice': dice,
        'iou': iou
    }

# Load and preprocess test data (same as before)
def load_and_preprocess_test_data(image_dir, mask_dir):
    image_paths = sorted([os.path.join(image_dir, f) for f in os.listdir(image_dir) if f.endswith('.png')])
    mask_paths = sorted([os.path.join(mask_dir, f) for f in os.listdir(mask_dir) if f.endswith('.png')])

    images = []
    masks = []

    for img_path, mask_path in zip(image_paths, mask_paths):
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (INPUT_SHAPE[0], INPUT_SHAPE[1]))
        img = np.expand_dims(img, axis=-1)
        img = img.astype(np.float32) / 255.0

        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        mask = cv2.resize(mask, (INPUT_SHAPE[0], INPUT_SHAPE[1]), interpolation=cv2.INTER_NEAREST)
        mask = np.expand_dims(mask, axis=-1)
        mask = mask.astype(np.float32) / 255.0
        mask = (mask > 0.5).astype(np.float32)

        images.append(img)
        masks.append(mask)

    return np.array(images), np.array(mask_paths), np.array(masks)

# Visualize predictions (same as before)
def visualize_predictions(images, true_masks, pred_masks, num_samples=5):
    plt.figure(figsize=(15, 5*num_samples))

    for i in range(num_samples):
        plt.subplot(num_samples, 3, i*3 + 1)
        plt.imshow(images[i].squeeze(), cmap='gray')
        plt.title('Original Image')
        plt.axis('off')

        plt.subplot(num_samples, 3, i*3 + 2)
        plt.imshow(true_masks[i].squeeze(), cmap='gray')
        plt.title('True Mask')
        plt.axis('off')

        plt.subplot(num_samples, 3, i*3 + 3)
        plt.imshow(pred_masks[i].squeeze(), cmap='gray')
        plt.title('Predicted Mask')
        plt.axis('off')

    plt.tight_layout()
    plt.show()

# Main test function
def test_model():
    # Download model
    print("Downloading model from GitHub...")
    model_path = os.path.join(LOCAL_DIR, "best_model.h5")
    download_from_github(MODEL_URL, model_path)

    # Download and setup test data
    setup_test_data()

    # Load the trained model
    model = tf.keras.models.load_model(model_path, custom_objects={
        'dice_coef': dice_coef,
        'iou_coef': iou_coef,
        'dice_loss': lambda y_true, y_pred: 1 - dice_coef(y_true, y_pred),
        'bce_dice_loss': lambda y_true, y_pred: tf.keras.losses.binary_crossentropy(y_true, y_pred) + (1 - dice_coef(y_true, y_pred))
    })

    # Load test data
    test_images, test_mask_paths, test_masks = load_and_preprocess_test_data(TEST_IMAGE_DIR, TEST_MASK_DIR)

    # Make predictions
    predictions = model.predict(test_images)

    # Calculate metrics
    metrics = calculate_metrics(test_masks, predictions)
    print("\nTest Metrics:")
    print(f"Accuracy: {metrics['accuracy']:.4f}")
    print(f"Sensitivity (Recall): {metrics['sensitivity']:.4f}")
    print(f"Specificity: {metrics['specificity']:.4f}")
    print(f"Dice Coefficient: {metrics['dice']:.4f}")
    print(f"IoU (Jaccard Index): {metrics['iou']:.4f}")

    # Visualize some predictions
    print(f"\nVisualizing {NUM_SAMPLES_TO_VISUALIZE} predictions...")
    visualize_predictions(
        test_images[:NUM_SAMPLES_TO_VISUALIZE],
        test_masks[:NUM_SAMPLES_TO_VISUALIZE],
        (predictions[:NUM_SAMPLES_TO_VISUALIZE] > 0.5).astype(np.float32)
    )

if __name__ == "__main__":
    test_model()

In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import cv2
import requests
import zipfile
import io
from skimage.measure import label, regionprops
import matplotlib.pyplot as plt

# Define custom metrics
def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = tf.keras.backend.flatten(y_true)
    y_pred_f = tf.keras.backend.flatten(y_pred)
    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) + smooth)

def dice_loss(y_true, y_pred):
    return 1 - dice_coef(y_true, y_pred)

def bce_dice_loss(y_true, y_pred):
    return tf.keras.losses.binary_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred)

# GitHub configuration
GITHUB_REPO = "https://github.com/Safwan-Ul-Islam/CSE465_Spring2025_Group-5/tree/main"  # Replace with your GitHub repo
MODEL_URL = f"{GITHUB_REPO}/raw/main/Final%20project/best_model.h5"  # Path to model in repo
CSV_URL = f"{GITHUB_REPO}/raw/main/Final%20projectdataset/pixel_size_and_HC.csv"  # Path to CSV in repo
TEST_DATA_URL = f"{GITHUB_REPO}/archive/main.zip"  # Will download entire repo as zip

# Local paths
LOCAL_DIR = "./github_data"
MODEL_PATH = os.path.join(LOCAL_DIR, "Final project/best_model.h5")
TEST_IMAGE_DIR = os.path.join(LOCAL_DIR, "Final project/dataset/dataset/test_set/images")  # Adjust based on your repo structure
CSV_PATH = os.path.join(LOCAL_DIR, "Final project/dataset/pixel_size_and_HC.csv")
OUTPUT_DIR = os.path.join(LOCAL_DIR, "Final project/results")
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Download helper function
def download_from_github(url, save_path):
    response = requests.get(url)
    if response.status_code == 200:
        with open(save_path, 'wb') as f:
            f.write(response.content)
    else:
        raise Exception(f"Failed to download from {url}")

# Download and extract dataset
def setup_test_data():
    if not os.path.exists(LOCAL_DIR):
        os.makedirs(LOCAL_DIR)

    # Download the model
    print("Downloading model from GitHub...")
    download_from_github(MODEL_URL, MODEL_PATH)

    # Download the CSV
    print("Downloading CSV from GitHub...")
    download_from_github(CSV_URL, CSV_PATH)

    # Download the test images (entire repo as zip)
    print("Downloading test data from GitHub...")
    response = requests.get(TEST_DATA_URL)
    zip_file = zipfile.ZipFile(io.BytesIO(response.content))
    zip_file.extractall(LOCAL_DIR)

    print("All data downloaded and extracted.")

# Function to calculate head circumference from mask
def calculate_hc_from_mask(mask, pixel_size):
    labeled_mask = label(mask)
    regions = regionprops(labeled_mask)

    if len(regions) == 0:
        return 0.0

    largest_region = max(regions, key=lambda x: x.area)
    perimeter = largest_region.perimeter
    circumference_mm = perimeter * pixel_size

    return circumference_mm

# Main processing function
def process_hc_measurements():
    # Setup data from GitHub
    setup_test_data()

    # Load the trained model with custom objects
    try:
        model = tf.keras.models.load_model(
            MODEL_PATH,
            custom_objects={
                'dice_coef': dice_coef,
                'dice_loss': dice_loss,
                'bce_dice_loss': bce_dice_loss
            }
        )
    except Exception as e:
        print(f"Error loading model: {e}")
        model = tf.keras.models.load_model(MODEL_PATH)

    # Load the CSV with ground truth measurements
    df = pd.read_csv(CSV_PATH)
    df['filename'] = df['filename'].apply(lambda x: os.path.basename(x))

    # Process test images and compare with ground truth
    results = []
    for filename in os.listdir(TEST_IMAGE_DIR):
        if filename.endswith('.png'):
            csv_row = df[df['filename'] == filename]
            if len(csv_row) == 0:
                print(f"No CSV entry found for {filename}, skipping...")
                continue

            pixel_size = csv_row['pixel size(mm)'].values[0]
            true_hc = csv_row['head circumference (mm)'].values[0]

            img_path = os.path.join(TEST_IMAGE_DIR, filename)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if img is None:
                print(f"Could not read image {filename}, skipping...")
                continue

            img = cv2.resize(img, (256, 256))
            img = np.expand_dims(img, axis=-1)
            img = img.astype(np.float32) / 255.0
            img = np.expand_dims(img, axis=0)

            try:
                pred_mask = model.predict(img, verbose=0)[0]
                pred_mask = (pred_mask > 0.5).astype(np.uint8)

                pred_hc = calculate_hc_from_mask(pred_mask.squeeze(), pixel_size)

                error_mm = abs(pred_hc - true_hc)
                error_percent = (error_mm / true_hc) * 100 if true_hc != 0 else 0

                results.append({
                    'filename': filename,
                    'true_hc': true_hc,
                    'pred_hc': pred_hc,
                    'error_mm': error_mm,
                    'error_percent': error_percent,
                    'pixel_size': pixel_size
                })
            except Exception as e:
                print(f"Error processing {filename}: {e}")

    # Convert results to DataFrame
    if results:
        results_df = pd.DataFrame(results)

        # Calculate overall accuracy metrics
        mean_error_mm = results_df['error_mm'].mean()
        mean_error_percent = results_df['error_percent'].mean()
        std_error_mm = results_df['error_mm'].std()
        median_error_mm = results_df['error_mm'].median()

        print("\nHead Circumference Measurement Results:")
        print(f"Number of samples analyzed: {len(results_df)}")
        print(f"Mean Absolute Error: {mean_error_mm:.2f} mm")
        print(f"Median Absolute Error: {median_error_mm:.2f} mm")
        print(f"Mean Percentage Error: {mean_error_percent:.2f}%")
        print(f"Standard Deviation: {std_error_mm:.2f} mm")

        # Save results to CSV
        results_csv_path = os.path.join(OUTPUT_DIR, 'hc_measurement_results.csv')
        results_df.to_csv(results_csv_path, index=False)
        print(f"\nResults saved to: {results_csv_path}")

        # Visualize some examples with measurements
        num_samples = min(5, len(results_df))
        plt.figure(figsize=(15, 3*num_samples))

        for i, (_, row) in enumerate(results_df.head(num_samples).iterrows()):
            img_path = os.path.join(TEST_IMAGE_DIR, row['filename'])
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

            img_processed = cv2.resize(img, (256, 256))
            img_processed = np.expand_dims(img_processed, axis=-1)
            img_processed = img_processed.astype(np.float32) / 255.0
            img_processed = np.expand_dims(img_processed, axis=0)
            pred_mask = model.predict(img_processed, verbose=0)[0]
            pred_mask = (pred_mask > 0.5).astype(np.uint8)

            plt.subplot(num_samples, 2, 2*i+1)
            plt.imshow(img, cmap='gray')
            plt.title(f"Original: {row['filename']}\nTrue HC: {row['true_hc']:.1f}mm")
            plt.axis('off')

            plt.subplot(num_samples, 2, 2*i+2)
            plt.imshow(pred_mask.squeeze(), cmap='gray')
            plt.title(f"Predicted HC: {row['pred_hc']:.1f}mm\nError: {row['error_mm']:.1f}mm ({row['error_percent']:.1f}%)")
            plt.axis('off')

        plt.tight_layout()
        plt.savefig(os.path.join(OUTPUT_DIR, 'hc_comparison.png'))
        plt.show()
    else:
        print("No valid results were generated. Please check your input paths and data.")

if __name__ == "__main__":
    process_hc_measurements()

In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import cv2
import requests
import zipfile
import io
from skimage.measure import label, regionprops
import matplotlib.pyplot as plt
from tabulate import tabulate

# GitHub configuration
GITHUB_REPO = "https://github.com/Safwan-Ul-Islam/CSE465_Spring2025_Group-5/tree/main"  # Replace with your GitHub repo
MODEL_URL = f"{GITHUB_REPO}/raw/main/Final%20project/best_model.h5"  # Path to model in repo
CSV_URL = f"{GITHUB_REPO}/raw/main/Final%20project/dataset/pixel_size_and_HC.csv"  # Path to CSV in repo
TEST_DATA_URL = f"{GITHUB_REPO}/archive/main.zip"  # Will download entire repo as zip

# Local paths
LOCAL_DIR = "./github_data"
MODEL_PATH = os.path.join(LOCAL_DIR, "Final project/best_model.h5")
TEST_IMAGE_DIR = os.path.join(LOCAL_DIR, "Final project/dataset/test_set/images")  # Adjust based on your repo structure
CSV_PATH = os.path.join(LOCAL_DIR, "Final project/dataset/pixel_size_and_HC.csv")
OUTPUT_DIR = os.path.join(LOCAL_DIR, "Final project/results")
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Define all custom objects that might be in the model
def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = tf.keras.backend.flatten(y_true)
    y_pred_f = tf.keras.backend.flatten(y_pred)
    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) + smooth)

def dice_loss(y_true, y_pred):
    return 1 - dice_coef(y_true, y_pred)

def bce_dice_loss(y_true, y_pred):
    return tf.keras.losses.binary_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred)

def download_from_github(url, save_path):
    """Download a file from GitHub"""
    response = requests.get(url)
    if response.status_code == 200:
        with open(save_path, 'wb') as f:
            f.write(response.content)
    else:
        raise Exception(f"Failed to download from {url}")

def setup_data():
    """Download all required data from GitHub"""
    if not os.path.exists(LOCAL_DIR):
        os.makedirs(LOCAL_DIR)

    # Download model
    print("Downloading model...")
    download_from_github(MODEL_URL, MODEL_PATH)

    # Download CSV
    print("Downloading CSV...")
    download_from_github(CSV_URL, CSV_PATH)

    # Download test images (entire repo as zip)
    print("Downloading test images...")
    response = requests.get(TEST_DATA_URL)
    zip_file = zipfile.ZipFile(io.BytesIO(response.content))
    zip_file.extractall(LOCAL_DIR)

    print("All data downloaded successfully")

def calculate_hc(mask, pixel_size):
    """Calculate head circumference from binary mask"""
    mask = mask.squeeze().astype(np.uint8)
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        return 0.0
    largest_contour = max(contours, key=cv2.contourArea)
    perimeter = cv2.arcLength(largest_contour, True)
    return perimeter * pixel_size

def main():
    # Setup data from GitHub
    setup_data()

    # Load model with multiple fallback options
    try:
        model = tf.keras.models.load_model(
            MODEL_PATH,
            custom_objects={
                'dice_coef': dice_coef,
                'dice_loss': dice_loss,
                'bce_dice_loss': bce_dice_loss
            }
        )
    except:
        try:
            model = tf.keras.models.load_model(MODEL_PATH, compile=False)
            model.compile(optimizer='adam', loss=bce_dice_loss, metrics=[dice_coef])
        except Exception as e:
            print(f"Failed to load model: {e}")
            raise

    # Load ground truth data
    df = pd.read_csv(CSV_PATH)
    df['filename'] = df['filename'].apply(lambda x: os.path.basename(x))

    # Process all test images
    results = []
    for filename in os.listdir(TEST_IMAGE_DIR):
        if not filename.endswith('.png'):
            continue

        # Find matching CSV entry
        csv_match = df[df['filename'] == filename]
        if csv_match.empty:
            print(f"No CSV match for {filename}")
            continue

        pixel_size = csv_match['pixel size(mm)'].values[0]
        true_hc = csv_match['head circumference (mm)'].values[0]

        # Load and preprocess image
        img_path = os.path.join(TEST_IMAGE_DIR, filename)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            print(f"Failed to load {filename}")
            continue

        img_proc = cv2.resize(img, (256, 256))
        img_proc = np.expand_dims(img_proc, axis=-1)
        img_proc = img_proc.astype(np.float32) / 255.0

        # Predict and calculate HC
        pred_mask = model.predict(np.expand_dims(img_proc, axis=0), verbose=0)[0]
        pred_hc = calculate_hc((pred_mask > 0.5).astype(np.uint8), pixel_size)

        results.append({
            'Filename': filename,
            'Pixel Size (mm)': f"{pixel_size:.4f}",
            'True HC (mm)': f"{true_hc:.2f}",
            'Predicted HC (mm)': f"{pred_hc:.2f}",
            'Abs Error (mm)': f"{abs(pred_hc - true_hc):.2f}",
            'Rel Error (%)': f"{100 * abs(pred_hc - true_hc) / true_hc:.2f}"
        })

    # Create and save results
    results_df = pd.DataFrame(results)
    results_df.to_csv(os.path.join(OUTPUT_DIR, 'hc_comparison_results.csv'), index=False)

    # Calculate statistics
    stats = {
        'Metric': ['Mean', 'Median', 'Std Dev', 'Max', 'Min'],
        'Absolute Error (mm)': [
            results_df['Abs Error (mm)'].astype(float).mean(),
            results_df['Abs Error (mm)'].astype(float).median(),
            results_df['Abs Error (mm)'].astype(float).std(),
            results_df['Abs Error (mm)'].astype(float).max(),
            results_df['Abs Error (mm)'].astype(float).min()
        ],
        'Relative Error (%)': [
            results_df['Rel Error (%)'].astype(float).mean(),
            results_df['Rel Error (%)'].astype(float).median(),
            results_df['Rel Error (%)'].astype(float).std(),
            results_df['Rel Error (%)'].astype(float).max(),
            results_df['Rel Error (%)'].astype(float).min()
        ]
    }
    stats_df = pd.DataFrame(stats)
    stats_df.to_csv(os.path.join(OUTPUT_DIR, 'hc_error_statistics.csv'), index=False)

    # Display results
    print("\n=== HC MEASUREMENT COMPARISON ===")
    print(tabulate(results_df, headers='keys', tablefmt='grid', showindex=False))

    print("\n=== ERROR STATISTICS ===")
    print(tabulate(stats_df, headers='keys', tablefmt='grid', showindex=False))

    # Visualization
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.scatter(
        results_df['True HC (mm)'].astype(float),
        results_df['Predicted HC (mm)'].astype(float),
        alpha=0.6
    )
    plt.plot([0, max(results_df['True HC (mm)'].astype(float))],
             [0, max(results_df['True HC (mm)'].astype(float))], 'r--')
    plt.xlabel('True HC (mm)')
    plt.ylabel('Predicted HC (mm)')
    plt.title('True vs Predicted HC')
    plt.grid(True)

    plt.subplot(1, 2, 2)
    plt.hist(results_df['Abs Error (mm)'].astype(float), bins=20, alpha=0.7)
    plt.xlabel('Absolute Error (mm)')
    plt.ylabel('Frequency')
    plt.title('Error Distribution')
    plt.grid(True)

    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_DIR, 'hc_comparison_plots.png'))
    plt.show()

if __name__ == "__main__":
    main()